<a href="https://colab.research.google.com/github/skalce0/demo/blob/main/IFapp22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark
from pyspark import SparkConf
from pyspark.context import SparkContext
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 44 kB/s 
     |████████████████████████████████| 199 kB 57.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=ac29b66a40a104a6cf77144abf9d51134698982388a645a88a64ef1a9c35b443
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [ ]:
L = [3,4,3,5,6,7,8,1,1,0]

In [ ]:
n =sc.parallelize(L,4)
# si m est le nombre de cores à disposition pour Spark
# le partitionnement est sur k*m parties où 1<=k<=4

In [ ]:
n

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

In [ ]:
n.count()

10

In [ ]:
n.collect()

[3, 4, 3, 5, 6, 7, 8, 1, 1, 0]

In [ ]:
n.take(5)

[3, 4, 3, 5, 6]

In [ ]:
p = n.glom()

In [ ]:
p

PythonRDD[2] at RDD at PythonRDD.scala:53

In [ ]:
p.count()

4

In [ ]:
p.collect()

[[3, 4], [3, 5], [6, 7], [8, 1, 1, 0]]

In [ ]:
s = n.map(lambda x : x +1)

In [ ]:
s.collect()

[4, 5, 4, 6, 7, 8, 9, 2, 2, 1]

In [ ]:
s.glom().collect()

[[4, 5], [4, 6], [7, 8], [9, 2, 2, 1]]

In [ ]:
s.reduce(lambda x,y: x+y)

48

In [ ]:
# calculer la sommes des nombres pairs et la somme des nombres impairs
# dans s

In [ ]:
# prenez quelques minutes pour refleshir à une possible solution avec code


In [ ]:
s.map(lambda x : (x%2, x)).reduceByKey(lambda x,y:x+y).collect()

[(0, 26), (1, 22)]

In [ ]:
# reduceByKey effectue : 
# un shuffle --> (0, [4,4,6,8,2,2]) et (1, [5,7,9,1])
# et l'aggregation sur les valeurs obtenues par groupement pour chaque cle, 
# via une fontion lambda 

In [ ]:
# 0 \subset= S join T \subset= S X T 

In [ ]:
l=sc.parallelize([("a",1), ("a",2), ("a",3)])
r=sc.parallelize([("a",4), ("a",2), ("a",1)])
l.join(r).collect()

[('a', (1, 4)),
 ('a', (1, 2)),
 ('a', (1, 1)),
 ('a', (2, 4)),
 ('a', (2, 2)),
 ('a', (2, 1)),
 ('a', (3, 4)),
 ('a', (3, 2)),
 ('a', (3, 1))]

In [ ]:
# jointures en Spark 

In [ ]:
l=sc.parallelize([("a",1), ("a",2), ("b",3)])
r=sc.parallelize([("a",4), ("c",2), ("b",1)])
l.join(r).collect()

[('b', (3, 1)), ('a', (1, 4)), ('a', (2, 4))]

In [ ]:
# calculez les moyennes des nombres pairs/impairs. <---

In [ ]:
snpi = s.map(lambda x : (x%2, x)).reduceByKey(lambda x,y:x+y)

In [ ]:
snpi.collect()

[(0, 26), (1, 22)]

In [ ]:
cnpi = s.map(lambda x : (x%2, 1)).reduceByKey(lambda x,y:x+y)

In [ ]:
cnpi.collect()

[(0, 6), (1, 4)]

In [ ]:
snpi.join(cnpi).map(lambda x : (x[0], x[1][0]/x[1][1])).collect()

[(0, 4.333333333333333), (1, 5.5)]

In [ ]:
# autre solution
snpi.join(cnpi).mapValues(lambda x: x[0] / x[1]).collect()



[(0, 4.333333333333333), (1, 5.5)]

In [ ]:
# flatMap
n.collect()

[3, 4, 3, 5, 6, 7, 8, 1, 1, 0]

In [ ]:
n.map(lambda x : list(range(x))).collect()

[[0, 1, 2],
 [0, 1, 2, 3],
 [0, 1, 2],
 [0, 1, 2, 3, 4],
 [0, 1, 2, 3, 4, 5],
 [0, 1, 2, 3, 4, 5, 6],
 [0, 1, 2, 3, 4, 5, 6, 7],
 [0],
 [0],
 []]

In [ ]:
n.flatMap(lambda x : list(range(x))).collect()

[0,
 1,
 2,
 0,
 1,
 2,
 3,
 0,
 1,
 2,
 0,
 1,
 2,
 3,
 4,
 0,
 1,
 2,
 3,
 4,
 5,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 0,
 0]

In [ ]:
n.filter(lambda x : x%2==0).collect()

[4, 6, 8, 0]

In [ ]:
# encodez filter en terme de flatMap
#n.flatMap( )

n.flatMap(lambda x: [x] if x%2==0 else []).collect()

[4, 6, 8, 0]

In [ ]:
n.collect()

[3, 4, 3, 5, 6, 7, 8, 1, 1, 0]

In [ ]:
n.reduce(lambda x,y:x+y)

38

In [ ]:
spi = n.map(lambda x:(x%2, x)).reduceByKey(lambda x,y: x +y)

In [ ]:
spi.collect()

[(0, 18), (1, 20)]

In [ ]:
cpi =  n.map(lambda x:(x%2, 1)).reduceByKey(lambda x,y: x +y)

In [ ]:
cpi.collect()

[(0, 4), (1, 6)]

In [ ]:
spi.join(cpi).map(lambda x: (x[0], x[1][0]/x[1][1])).collect()

[(0, 4.5), (1, 3.3333333333333335)]

In [ ]:
# version optimisée

In [ ]:
scpi=n.map(lambda x:(x%2,(x,1))).reduceByKey(lambda x,y: (x[0]+y[0], x[1] +y[1]))

In [ ]:
scpi.map(lambda x: (x[0], x[1][0]/x[1][1])).collect()

[(0, 4.5), (1, 3.3333333333333335)]

In [ ]:
# word count

In [ ]:
text = sc.parallelize(["aa bb vv", "gg aa vv", "bb aa gg" ],10)

In [ ]:
text.collect()

['aa bb vv', 'gg aa vv', 'bb aa gg']

In [ ]:
text.glom().collect()

[[], [], [], ['aa bb vv'], [], [], ['gg aa vv'], [], [], ['bb aa gg']]

In [ ]:
def wordcount(t):
  wordsone =t.flatMap(lambda l : [(w,1) for w in  l.split()])
  return wordsone.reduceByKey(lambda x,y: x + y)


In [ ]:
wordcount(text).collect()

[('bb', 2), ('gg', 2), ('aa', 3), ('vv', 2)]

In [ ]:
! wget https://www.dropbox.com/s/1t8an83nqolen0s/shake.txt

--2022-12-08 08:37:14--  https://www.dropbox.com/s/1t8an83nqolen0s/shake.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/1t8an83nqolen0s/shake.txt [following]
--2022-12-08 08:37:14--  https://www.dropbox.com/s/raw/1t8an83nqolen0s/shake.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucb9cb7009bfbdc38f1a5f61b509.dl.dropboxusercontent.com/cd/0/inline/ByNl9hXS_-_ejvLIrYhfm0r_VngJkDIJTwoXfIGJDh5DHyL5a5SQLQpF__7xuUdcDaFROhsot2Bvl1ELiRvSlOLB1mqp9UzlYGCmvfpPSaHHSPUgVG2PSo9NDHAtimgLctMvLo6cGmBOFL4tNb2E3x5Km0NWoeKAseV43-ApXAEERA/file# [following]
--2022-12-08 08:37:14--  https://ucb9cb7009bfbdc38f1a5f61b509.dl.dropboxusercontent.com/cd/0/inline/ByNl9hXS_-_ejvLIrYhfm0r_VngJkDIJTwoXfIGJDh5DHyL5a5SQLQpF__7xuUdcDaFROhsot2Bvl1ELiRvSlOLB1

In [ ]:
! ls


sample_data  shake.txt


In [ ]:
! pwd

/content


In [ ]:
tshake = sc.textFile("/content/shake.txt")

In [ ]:
tshake.count()

124465

In [ ]:
tshake.take(20)

['This is the 100th Etext file presented by Project Gutenberg, and',
 'is presented in cooperation with World Library, Inc., from their',
 'Library of the Future and Shakespeare CDROMS.  Project Gutenberg',
 'often releases Etexts that are NOT placed in the Public Domain!!',
 '',
 'Shakespeare',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '*This Etext has certain copyright implications you should read!*',
 '',
 '<<THIS ELECTRONIC VERSION OF THE COMPLETE WORKS OF WILLIAM']

In [ ]:
wordcount(tshake).count()

67505

In [ ]:
wordcount(tshake).take(30)

[('is', 7851),
 ('Etext', 4),
 ('presented', 11),
 ('Project', 13),
 ('Gutenberg,', 1),
 ('in', 9576),
 ('cooperation', 1),
 ('World', 5),
 ('Library,', 2),
 ('Inc.,', 1),
 ('of', 15544),
 ('Future', 3),
 ('Shakespeare', 45),
 ('are', 2917),
 ('placed', 10),
 ('Public', 1),
 ('Domain!!', 1),
 ('*This', 1),
 ('certain', 116),
 ('copyright', 7),
 ('implications', 1),
 ('read!*', 1),
 ('ELECTRONIC', 442),
 ('VERSION', 221),
 ('OF', 1490),
 ('WILLIAM', 244),
 ('SHAKESPEARE', 223),
 ('IS', 445),
 ('COPYRIGHT', 221),
 ('WORLD', 221)]

In [ ]:
# chercher les mots le plus frequents
wc = wordcount(tshake).persist()
frequences = wc.map(lambda c : c[1] ) 
maxfreq = frequences.reduce(lambda x,y : max(x,y))
wc.filter(lambda c : c[1]==maxfreq  ).collect()

[('the', 23242)]

In [ ]:
pets =sc.parallelize([("cat", 1), ("dog", 1), ("cat", 2), ("dog", 3),("dog", 2)  ])

In [ ]:
# version moins efficace
ps = pets.reduceByKey(lambda x, y : x +y)
pc = pets.map(lambda x:(x[0], 1) ).reduceByKey(lambda x, y : x +y)
ps.join(pc)....idem...collect()


[('cat', (3, 2)), ('dog', (6, 3))]

In [ ]:
pets.map(lambda c : (c[0], (c[1], 1))  ).\
reduceByKey(lambda a,b : ( a[0]+b[0], a[1]+b[1] )).\
map(lambda x: (x[0], x[1][0]/x[1][1])).collect()

[('cat', 1.5), ('dog', 2.0)]